In [172]:
from faker import Faker
from pymongo import MongoClient
from datetime import datetime
import pandas as pd
import random

In [173]:
uri = 'mongodb://localhost/'
db = MongoClient(uri)['vendas']

# a quantidade de usuários define também a quantidade representantes
QUANTIDADE_USUARIOS = 10

In [174]:
df_ori = pd.read_csv('/home/oscar/escola/dados/Sales_Product_Combined.csv')

In [175]:
# renomeando campos
df_aux = df_ori.rename(columns={
  'Order ID': 'id', 
  'Product' : 'produto', 
  'Quantity Ordered': 'quantidade', 
  'Price': 'preco', 
  'Order Date': 'data',
  'Time': 'hora',
  'Product Type': 'categoria', 
  'Purchase Address': 'endereco', 
  'City': 'cidade',
})

# criando campo horário
df_aux['horario'] = pd.to_datetime(df_aux['data'] + ' ' + df_aux['hora'], format='%d-%m-%Y %I:%M %p')

# diminuindo quantidade de registros
df_aux['cidade'] = df_aux['cidade'].str.strip()
# df_aux = df_aux.query('horario.dt.year == 2019 and cidade == "San Francisco"')[
df_aux = df_aux.query('horario.dt.year == 2019 and cidade == "Austin"')[
  ['horario', 'produto', 'quantidade', 'preco']
]

In [176]:
df = df_aux.sort_values('horario')

# campo auxiliar para aleatórios
df['rnd'] = [random.randrange(100) for _ in df.index]

# transferir 25% das vendas dos meses 3, 4 e 5 para os meses 9, 10 e 11
df['horario'] = df['horario'].mask(
  (df['horario'] > pd.to_datetime('01-03-2019', format='%d-%m-%Y')) & 
  (df['horario'] < pd.to_datetime('01-06-2019', format='%d-%m-%Y')) & 
  (df['rnd'] < 25), 
  df['horario'] + pd.DateOffset(months=6))

# transferir 20% das vendas do mês 12 para o mês 1
df['horario'] = df['horario'].mask(
  (df['horario'] > pd.to_datetime('01-12-2019', format='%d-%m-%Y')) & 
  (df['rnd'] < 20), 
  df['horario'] + pd.DateOffset(months=-11))

# distribuir aleatóriamente os registros em 5 anos
df['rnd'] = [random.randrange(100) for _ in df.index]
df['horario'] = df['horario'].mask(
  (df.rnd < 25), 
  df['horario'] + pd.DateOffset(years=2))
df['horario'] = df['horario'].mask(
  (df.rnd >= 25) & 
  (df.rnd < 50),
  df['horario'] + pd.DateOffset(years=3))
df['horario'] = df['horario'].mask(
  (df.rnd >= 50) & 
  (df.rnd < 75),
  df['horario'] + pd.DateOffset(years=4))
df['horario'] = df['horario'].mask(
  (df.rnd >= 75),
  df['horario'] + pd.DateOffset(years=5))

# campo auxiliar de data
df['data'] = pd.to_datetime(df['horario']).dt.date

# distribuir os itens do pedido
df['nota'] = df['horario'].dt.strftime('%Y%m%d-%M').str.slice(stop=10)

# permitir cálculo com o preço
df['preco'] = df['preco'].str.replace(',', '').astype(float)
df['preco'] = df.apply(lambda row: row['preco'] * 10 if row['preco'] < 80 else row['preco'], axis=1)
df['preco'] = df['preco'].round(2)

# calcular aleatóriamente quantidade, mas valores menores tem peso maior
pop = [i + 1 for i in range(40)]
wei = [i * i + 1 for i in range(40, 0, -1)]
df['quantidade'] = random.choices(pop, wei, k=len(df.index))
df['quantidade'] = (df['quantidade'] / (df['quantidade'] * df['preco'] / 10000 + 1).astype(int)).astype(int)
df['total'] = df['quantidade'] * df['preco']
df['total'] = df['total'].round(2)

# renomear produtos
prod = {
  'AA Batteries (4-pack)': 'P01',
  'AAA Batteries (4-pack)': 'P02',
  'Lightning Charging Cable': 'P03',
  'USB-C Charging Cable': 'P04',
  'LG Dryer': 'P05',
  'Apple Airpods Headphones': 'P06',
  'Bose SoundSport Headphones': 'P07',
  'Wired Headphones': 'P08',
  'iPhone': 'P09',
  'Macbook Pro Laptop': 'P10',
  'ThinkPad Laptop': 'P11',
  'LG Washing Machine': 'P12',
  '20in Monitor': 'P13',
  '27in 4K Gaming Monitor': 'P14',
  '27in FHD Monitor': 'P15',
  '34in Ultrawide Monitor': 'P16',
  'Google Phone': 'P17',
  'Vareebadd Phone': 'P18',
  'Flatscreen TV': 'P19',
}
df['produto'] = df.apply(lambda row: prod[row['produto']], axis=1)

df = df[['nota', 'data', 'horario', 'produto', 'quantidade']].sort_values('horario')

In [177]:
df.groupby(['produto'])['quantidade'].sum()

produto
P01    11665
P02    11724
P03    13320
P04    12477
P05      476
P06     9802
P07     7239
P08    11158
P09     2857
P10      951
P11     1306
P12      186
P13     2631
P14     3088
P15     4001
P16     3315
P17     2279
P18     1002
P19     2653
Name: quantidade, dtype: int64

In [178]:
df.head()

,nota,data,horario,produto,quantidade
75315,20210101-1,2021-01-01,2021-01-01 10:12:00,P04,9
76286,20210101-1,2021-01-01,2021-01-01 13:10:00,P08,1
40253,20210101-3,2021-01-01,2021-01-01 20:38:00,P06,6
72608,20210101-1,2021-01-01,2021-01-01 23:18:00,P02,25
67603,20210102-2,2021-01-02,2021-01-02 01:26:00,P03,11


In [179]:
b = df[['horario', 'quantidade']]
b.index = df.horario
b.resample(rule='ME', on='horario')['quantidade'].sum()

horario
2021-01-31    2028
2021-02-28    1809
2021-03-31    1670
2021-04-30    1580
2021-05-31    1875
2021-06-30    1884
2021-07-31    2083
2021-08-31    1681
2021-09-30    2255
2021-10-31    3671
2021-11-30    2998
2021-12-31    2857
2022-01-31    2237
2022-02-28    1429
2022-03-31    1345
2022-04-30    1949
2022-05-31    1415
2022-06-30    1941
2022-07-31    1928
2022-08-31    1817
2022-09-30    2152
2022-10-31    3613
2022-11-30    2892
2022-12-31    2856
2023-01-31    2041
2023-02-28    1706
2023-03-31    1637
2023-04-30    2137
2023-05-31    1408
2023-06-30    1941
2023-07-31    1750
2023-08-31    1576
2023-09-30    2230
2023-10-31    3419
2023-11-30    2531
2023-12-31    2498
2024-01-31    2402
2024-02-29    1715
2024-03-31    1882
2024-04-30    2113
2024-05-31    1816
2024-06-30    1494
2024-07-31    1991
2024-08-31    1518
2024-09-30    2106
2024-10-31    2973
2024-11-30    2586
2024-12-31    2695
Freq: ME, Name: quantidade, dtype: int64

In [180]:
xls = pd.read_excel('estoque.xlsx')
produtos = []
for i, e in enumerate(xls.to_dict('records')):
  produtos.append({
    'id': i + 1,
    'produto': {
      'marca': {
        'nome': e['marca'],
      },
      'categoria': {
        'titulo': e['categoria'],
      },
      'descricao': e['descricao'],
    },
    'quantidade': e['quantidade'],
    'preco_compra': e['preco_compra'],
    'preco_venda': e['preco_compra'] * 2,
  })

In [181]:
# df.query('horario > "2023-01-01" and horario < "2024-01-01"')
# df.query('data == "2023-01-01"')
teste = df.query('horario > "2023-01-01" and horario < "2023-01-02"').sort_values(['data', 'nota', 'produto'])

In [182]:
teste

,nota,data,horario,produto,quantidade
51148,20230101-0,2023-01-01,2023-01-01 22:06:00,P04,34
68358,20230101-0,2023-01-01,2023-01-01 23:08:00,P04,2
74264,20230101-0,2023-01-01,2023-01-01 11:00:00,P19,4
76404,20230101-1,2023-01-01,2023-01-01 19:11:00,P02,21
73463,20230101-1,2023-01-01,2023-01-01 14:12:00,P17,4
74670,20230101-2,2023-01-01,2023-01-01 15:27:00,P02,13
73698,20230101-3,2023-01-01,2023-01-01 13:30:00,P02,10
71977,20230101-3,2023-01-01,2023-01-01 10:32:00,P06,16
69596,20230101-3,2023-01-01,2023-01-01 15:39:00,P07,22
33341,20230101-4,2023-01-01,2023-01-01 14:41:00,P08,5


In [183]:
teste.shape

(10, 5)

In [184]:
representantes = list(db['venda_representante'].find())
representantes

[{'_id': ObjectId('6636a5a6bf67c4821d38e654'),
  'id': 2,
  'nome': 'Vitória da Mata',
  'cpf': '086.925.743-92',
  'nivel': 'CONFIRMAR',
  'user_id': 2,
  'clientes': [{'id': 1,
    'nome': 'MOURA S/A',
    'cnpj': '29.468.031/0001-53',
    'contato': 'Vitor Hugo Barbosa',
    'telefone': '+55 (079) 91711-6597',
    'email': 'catarina97@example.net',
    'endereco': {'logradouro': 'Colônia Nunes, 3',
     'bairro': 'Candelaria',
     'cidade': 'Souza',
     'estado': 'CE',
     'cep': '13642-215'}},
   {'id': 2,
    'nome': 'DA ROCHA S.A.',
    'cnpj': '85.731.240/0001-19',
    'contato': 'Sophie Sales',
    'telefone': '+55 91 98153 8215',
    'email': 'erodrigues@example.org',
    'endereco': {'logradouro': 'Conjunto de Pires',
     'bairro': 'Boa União 1ª Seção',
     'cidade': 'Porto da Mata',
     'estado': 'SE',
     'cep': '70639660'}},
   {'id': 3,
    'nome': 'DA CONCEIÇÃO PINTO E FILHOS - EI',
    'cnpj': '70.359.814/0001-00',
    'contato': 'Gabrielly Campos',
    'telefone

In [185]:
# estoques = list(db['venda_estoque'].find())

estoques = {}
for e in db['venda_estoque'].find():
  estoques[f"P{e['id']:02}"] = e
estoques


{'P01': {'_id': ObjectId('66399227489da24ed18e99c7'),
  'id': 1,
  'produto': {'marca': {'nome': 'Bot-Art'},
   'categoria': {'titulo': 'Xicaras'},
   'descricao': 'Leiteira Texas Media 550ml'},
  'quantidade': 1000,
  'preco_compra': 14.925,
  'preco_venda': 29.85},
 'P02': {'_id': ObjectId('66399227489da24ed18e99c8'),
  'id': 2,
  'produto': {'marca': {'nome': 'Bot-Art'},
   'categoria': {'titulo': 'Xicaras'},
   'descricao': 'Xicara Café c/ pires Veneza'},
  'quantidade': 1000,
  'preco_compra': 7.345,
  'preco_venda': 14.69},
 'P03': {'_id': ObjectId('66399227489da24ed18e99c9'),
  'id': 3,
  'produto': {'marca': {'nome': 'Renata'},
   'categoria': {'titulo': 'Biscoitos, Bolos e Torradas em Sache'},
   'descricao': 'Sache Biscoito AMANTEIGADO COCO RENATA SACHÊ 9G 2X2 CAIXA 280 und'},
  'quantidade': 1000,
  'preco_compra': 43.935,
  'preco_venda': 87.87},
 'P04': {'_id': ObjectId('66399227489da24ed18e99ca'),
  'id': 4,
  'produto': {'marca': {'nome': 'Renata'},
   'categoria': {'tit

In [187]:
# from pprint import pprint

itens_pedido = []
id = 0
nota = teste.iloc[0, 0]
for index, row in teste.iterrows():
  if nota != row['nota']:
    # pprint(itens_pedido)
    print(itens_pedido)
    itens_pedido = []
    id = 0
    nota = row['nota']
    # print(estoques[row['produto']])
    # print(row['nota'], row['data'], row['horario'], row['produto'], row['quantidade'])

  id += 1
  estoque = estoques[row['produto']]
  item = {
    "nota": nota,
    "id": id,
    "produto": estoque['produto'],
    "preco_compra": estoque['preco_compra'],
    "preco_venda": estoque['preco_venda'],
    "quantidade": row['quantidade'],
    "total": round(row['quantidade'] * estoque['preco_venda'], 2)
  }
  itens_pedido.append(item)
  # print(item)
# pprint(itens_pedido)
print(itens_pedido)


[{'nota': '20230101-0', 'id': 1, 'produto': {'marca': {'nome': 'Renata'}, 'categoria': {'titulo': 'Biscoitos, Bolos e Torradas em Sache'}, 'descricao': 'Sache de Biscoito Maria RENATA 11G CAIXA 180 UNIDADES'}, 'preco_compra': 34.1, 'preco_venda': 68.2, 'quantidade': 34, 'total': 2318.8}, {'nota': '20230101-0', 'id': 2, 'produto': {'marca': {'nome': 'Renata'}, 'categoria': {'titulo': 'Biscoitos, Bolos e Torradas em Sache'}, 'descricao': 'Sache de Biscoito Maria RENATA 11G CAIXA 180 UNIDADES'}, 'preco_compra': 34.1, 'preco_venda': 68.2, 'quantidade': 2, 'total': 136.4}, {'nota': '20230101-0', 'id': 3, 'produto': {'marca': {'nome': 'Atlântica'}, 'categoria': {'titulo': 'Toalha Para Hotel'}, 'descricao': 'Kit 20 Toalhas De Rosto Para Hotel, Motel, Pousada'}, 'preco_compra': 176.5, 'preco_venda': 353.0, 'quantidade': 4, 'total': 1412.0}, {'nota': '20230101-0', 'id': 4, 'produto': {'marca': {'nome': 'Bot-Art'}, 'categoria': {'titulo': 'Xicaras'}, 'descricao': 'Xicara Café c/ pires Veneza'}, 